# RPLib
## NCAA Men's Basketball Dataset, Big 12

1. We need a team list for every year. We need to adapt the code below so it handles a dynamic team list for each year.
2. There is an overall date range:
   (selection_sunday - 35 TO selection_sunday - 7) of games to consider. but within that range, let's move one game at a time. 

In [1]:
major_description = "Study of Big 12"
print(major_description)

Study of Big 12


In [2]:
%load_ext autoreload
%autoreload 2 
%matplotlib inline

In [3]:
import copy
import os
import pandas as pd
import numpy as np
import networkx as nx
from scipy.stats import pearsonr
from scipy.stats import skew
from tqdm import tqdm
import matplotlib.pyplot as plt
from joblib import Parallel, delayed
import itertools
import joblib
from scipy import stats

In [4]:
from pathlib import Path
home = str(Path.home())
home

'/home/jupyter-pander14'

In [5]:
# pyt in .ipython

In [6]:
import pyrankability

In [7]:
import marchmadness_study.base as base

In [8]:
import pyrplib

In [211]:
games={}
remaining_games={}
madness_teams={}
all_teams={}
years = ["2012","2013","2014","2015","2016","2017","2018"] # "2002","2003","2004","2005","2006","2007","2008","2009","2010","2011",
for year in years:
    games[year],remaining_games[year] = base.read_data(f'{home}/marchmadness_study/data/%steams.txt'%year,f'{home}/marchmadness_study/data/%sgames.txt'%year,f'{home}/marchmadness_study/data/%sMadnessTeams.txt'%year)
    madness_teams[year] = list(np.unique(list(games[year].team1_name.loc[games[year].team1_madness == 1]) + list(games[year].team2_name.loc[games[year].team2_madness == 1])))
    all_teams[year] = list(np.unique(list(games[year].team1_name) + list(games[year].team2_name)))
print(year)
games[year]

2018


,team2,team1,notsure1,date,H_A_N1,points1,H_A_N2,points2,team1_name,team2_name,team1_madness,team2_madness
4147,256,170,737008,2017-11-10,1,92,-1,77,Minnesota,SC_Upstate,0,0
4288,265,293,737008,2017-11-10,1,75,-1,50,Texas_Tech,South_Alabama,1,0
925,56,326,737008,2017-11-10,1,75,-1,60,Villanova,Columbia,1,0
521,32,176,737008,2017-11-10,1,79,-1,78,Monmouth_NJ,Bucknell,0,1
4252,263,235,737008,2017-11-10,1,105,-1,74,Purdue,SIUE,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...
1985,121,152,737122,2018-03-04,0,65,0,49,Loyola-Chicago,Illinois_St,1,0
933,57,114,737122,2018-03-04,1,81,-1,71,Houston,Connecticut,1,0
1370,81,164,737122,2018-03-04,1,90,-1,70,Memphis,East_Carolina,0,0
1483,88,143,737122,2018-03-04,-1,108,1,96,Lipscomb,FL_Gulf_Coast,1,0


## What does a dataset look like?

In [212]:
remaining_games[year].head()

,team2,team1,notsure1,date,H_A_N1,points1,H_A_N2,points2,team1_name,team2_name,team1_madness,team2_madness
2,1,73,737130,2018-03-12,1,80,-1,73,Drake,Abilene_Chr,0,0
32,2,313,737125,2018-03-07,1,97,-1,90,UNLV,Air_Force,0,0
42,3,79,737126,2018-03-08,0,67,0,58,E_Michigan,Akron,0,0
59,4,137,737128,2018-03-10,0,86,0,63,Kentucky,Alabama,1,1
68,4,326,737135,2018-03-17,0,81,0,58,Villanova,Alabama,1,1


## Select just the teams from the conferece

In [213]:
teams_by_year = {}
for year in years:
    #acc_teams_by_year[year] = ["Duke","Miami_FL","Wake_Forest","Boston_College","Notre_Dame","Pittsburgh","Virginia","Florida_St","Virginia_Tech","Georgia_Tech","Clemson","North_Carolina","Louisville","Syracuse","NC_State"]
    teams_by_year[year] = ["TCU","Baylor","Iowa_St","Kansas","Kansas_St","Oklahoma","Oklahoma_St","Texas","Texas_Tech","West_Virginia"]

## Restrict to games where one of these teams played at least

In [214]:
for year, acc_teams in teams_by_year.items():
    team1_name = games[year].team1_name
    team2_name = games[year].team2_name
    games[year] = games[year].loc[team1_name.isin(acc_teams) | team2_name.isin(acc_teams)]
    
    team1_name = remaining_games[year].team1_name
    team2_name = remaining_games[year].team2_name
    remaining_games[year] = remaining_games[year].loc[team1_name.isin(acc_teams) | team2_name.isin(acc_teams)]

In [215]:
len(games['2018'])

217

## Now compute the final standings for each year based on winning percentage

In [216]:
pyrplib.transformers.ComputeDTransformer?

In [217]:
percentages = {}
wins = {}
loses = {}
for year in years:
    games_df = pd.DataFrame({"team1_name":games[year]['team1_name'],
                            "team1_score":games[year]['points1'],
                            "team1_H_A_N": games[year]['H_A_N1'],
                            "team2_name":games[year]['team2_name'],
                            "team2_score":games[year]['points2'],
                            "team2_H_A_N": games[year]['H_A_N1'],
                            "date": games[year]['date']
                           }).sort_values(by='date').drop('date',axis=1)    
    D,ID = pyrplib.transformers.ComputeDTransformer(team_range=teams_by_year[year]).fit_transform(games_df)
    D = D.fillna(0)
    win = pd.Series(0,index=D.index)
    lose = pd.Series(0,index=D.index)
    for i in range(len(D.index)):
        for j in range(i+1,len(D.index)):
            team1 = D.index[i]
            team2 = D.index[j]
            win.loc[team1] += D.loc[team1,team2]
            lose.loc[team1] += D.loc[team2,team1]
            win.loc[team2] += D.loc[team2,team1]
            lose.loc[team2] += D.loc[team1,team2]
    percentage = win*1./(win+lose) #win.loc['Kansas'],lose.loc['Kansas']
    wins[year] = win
    loses[year] = lose
    percentages[year] = percentage

In [218]:
percentages['2018'].sort_values()

team1
Iowa_St          0.222222
Baylor           0.444444
Oklahoma         0.444444
Oklahoma_St      0.444444
Texas            0.444444
TCU              0.500000
Kansas_St        0.555556
Texas_Tech       0.611111
West_Virginia    0.611111
Kansas           0.722222
dtype: float64

In [219]:
# Identify games that aren't consistent with this

In [268]:
consistent_games = {}
inconsistent_games = {}
for year, acc_teams in teams_by_year.items():
    print(year)
    team1_name = games[year].team1_name
    team2_name = games[year].team2_name
    direct = team1_name.isin(acc_teams) & team2_name.isin(acc_teams)
    games[year].loc['direct'] = direct
    game_ixs = np.where(direct)[0]
    for ix in game_ixs:
        team1 = team1_name.iloc[ix]
        team2 = team2_name.iloc[ix]
        cons = False
        if games[year]['points1'].iloc[ix] > games[year]['points2'].iloc[ix] and percentages[year].loc[team1] > percentages[year].loc[team2]:
            cons = True
        elif games[year]['points2'].iloc[ix] > games[year]['points1'].iloc[ix] and percentages[year].loc[team2] > percentages[year].loc[team1]:
            cons = True
        if cons:
            if year not in consistent_games:
                consistent_games[year] = games[year].iloc[[ix]].copy()
            else:
                consistent_games[year] = consistent_games[year].append(games[year].iloc[[ix]].copy())
        else:
            if year not in inconsistent_games:
                inconsistent_games[year] = games[year].iloc[[ix]].copy()
            else:
                inconsistent_games[year] = inconsistent_games[year].append(games[year].iloc[[ix]].copy())

2012
2013
2014
2015
2016
2017
2018


/opt/tljh/user/lib/python3.7/site-packages/pandas/core/indexing.py:1700: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, v, pi)


In [222]:
percentages['2018'].sort_values()

team1
Iowa_St          0.222222
Baylor           0.444444
Oklahoma         0.444444
Oklahoma_St      0.444444
Texas            0.444444
TCU              0.500000
Kansas_St        0.555556
Texas_Tech       0.611111
West_Virginia    0.611111
Kansas           0.722222
dtype: float64

In [267]:
for year in years:
    print(year)
    games1 = consistent_games[year].groupby('team1_name').apply(lambda x: len(x))
    games2 = consistent_games[year].groupby('team2_name').apply(lambda x: len(x))
    display(games1 + games2.reindex(index=games1.index).fillna(0))

2012


team1_name
Baylor           13.0
Iowa_St          10.0
Kansas           13.0
Kansas_St        11.0
Oklahoma         10.0
Oklahoma_St      11.0
TCU               1.0
Texas            11.0
West_Virginia     2.0
dtype: float64

2013


team1_name
Baylor           15.0
Iowa_St          13.0
Kansas           12.0
Kansas_St        14.0
Oklahoma         14.0
Oklahoma_St      13.0
Texas            13.0
Texas_Tech       17.0
West_Virginia    17.0
dtype: float64

2014


...

team1_name
Baylor           14.0
Iowa_St          12.0
Kansas           15.0
Kansas_St        15.0
Oklahoma         12.0
Oklahoma_St      15.0
Texas            11.0
Texas_Tech       13.0
West_Virginia    13.0
dtype: float64

2017


team1_name
Baylor           12.0
Iowa_St          11.0
Kansas           16.0
Kansas_St        12.0
Oklahoma         13.0
Oklahoma_St      15.0
TCU              13.0
Texas_Tech       11.0
West_Virginia     9.0
dtype: float64

2018


team1_name
Baylor            9.0
Kansas           13.0
Kansas_St        16.0
Oklahoma          6.0
Oklahoma_St       8.0
TCU              13.0
Texas             9.0
Texas_Tech       10.0
West_Virginia    12.0
dtype: float64

In [269]:
def compute(dt,game_df_sample,team_range,method):
    if method == 'colley' or method == 'massey':
        if method == 'colley':
            map_func = lambda linked: pyrankability.construct.colley_matrices(linked,direct_thres=dt)
        elif method == 'massey':
            map_func = lambda linked: pyrankability.construct.massey_matrices(linked,direct_thres=dt)

        matrix,b,indirect_matrix,indirect_b = pyrankability.construct.map_vectorized(game_df_sample,map_func)
        matrix = matrix.reindex(index=team_range,columns=team_range)
        b = b.reindex(team_range)
        indirect_matrix = indirect_matrix.reindex(index=team_range,columns=team_range)
        indirect_b = indirect_b.reindex(team_range)
        ranking1,r1,perm1 = pyrankability.rank.ranking_from_matrices(matrix.fillna(0),b.fillna(0))
        #indirect_ranking1,indirect_r1,indirect_perm1 = pyrankability.rank.ranking_from_matrices(indirect_matrix.fillna(0),indirect_b.fillna(0))
        #if iw > 0:
        #    r1 = r1+iw*indirect_r1
        #    perm1, ranking1 = pyrankability.rank.perm_ranking_from_r(r1)
    else:
        D,ID = pyrplib.transformers.ComputeDTransformer(team_range=team_range).fit_transform(game_df_sample)
        if method == 'lop':
            D = D.fillna(0)
            delta,details = pyrankability.rank.solve(D,method="lop",cont=True)
        elif method == 'hillside':
            delta,details = pyrankability.rank.solve(D,method="hillside",cont=False)
        print(year,method,delta)
        x = pd.DataFrame(details['x'],index=D.index,columns=D.columns)
        r1 = x.sum(axis=1)
        perm1,ranking1 = pyrankability.rank.perm_ranking_from_r(r1)
        #print('x')
        #display(x.iloc[perm1,:].iloc[:,perm1])
        print('D')
        display(D.iloc[perm1,:].iloc[:,perm1])
    return ranking1, r1, perm1

In [270]:
def get_results(consistent_games,dt = 0,methods = ['colley','massey','lop','hillside']):
    results = pd.DataFrame(columns=['ranking','r','perm','year','method']).set_index(['year','method'])
    for year, team_range in teams_by_year.items():
        games_df = pd.DataFrame({"team1_name":consistent_games[year]['team1_name'],
                                "team1_score":consistent_games[year]['points1'],
                                "team1_H_A_N": consistent_games[year]['H_A_N1'],
                                "team2_name":consistent_games[year]['team2_name'],
                                "team2_score":consistent_games[year]['points2'],
                                "team2_H_A_N": consistent_games[year]['H_A_N1'],
                                "date": consistent_games[year]['date']
                               }).sort_values(by='date').drop('date',axis=1)    

        for method in methods:
            ranking,r,perm = compute(dt,games_df,team_range,method)
            entry = pd.Series([ranking,r,perm],index=['ranking','r','perm'],name=(year,method))
            results = results.append(entry)
    return results

In [271]:
consistent_results = get_results(consistent_games)
consistent_results

2018 lop 0.0
D


team2,Kansas,Baylor,Iowa_St,West_Virginia,TCU,Texas,Kansas_St,Oklahoma_St,Oklahoma,Texas_Tech
team1,,,,,,,,,,
Kansas,0.0,2.0,1.0,0.0,0.0,2.0,2.0,2.0,2.0,2.0
Baylor,0.0,0.0,1.0,1.0,0.0,2.0,1.0,2.0,2.0,2.0
Iowa_St,0.0,0.0,0.0,0.0,0.0,1.0,2.0,1.0,2.0,2.0
West_Virginia,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
TCU,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
Texas,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,2.0,2.0
Kansas_St,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,2.0
Oklahoma_St,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,2.0
Oklahoma,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


2018 hillside 5
D


team2,Kansas,Iowa_St,West_Virginia,Baylor,Kansas_St,Texas,Oklahoma_St,Oklahoma,Texas_Tech,TCU
team1,,,,,,,,,,
Kansas,NaN,1.0,NaN,2.0,2.0,2.0,2.0,2.0,2.0,NaN
Iowa_St,0.0,NaN,NaN,0.0,2.0,1.0,1.0,2.0,2.0,NaN
West_Virginia,NaN,NaN,NaN,0.0,1.0,NaN,NaN,NaN,NaN,NaN
Baylor,0.0,1.0,1.0,NaN,1.0,2.0,2.0,2.0,2.0,NaN
Kansas_St,0.0,0.0,0.0,0.0,NaN,0.0,2.0,NaN,2.0,NaN
Texas,0.0,0.0,NaN,0.0,1.0,NaN,1.0,2.0,2.0,NaN
Oklahoma_St,0.0,0.0,NaN,0.0,0.0,0.0,NaN,1.0,2.0,NaN
Oklahoma,0.0,0.0,NaN,0.0,NaN,0.0,0.0,NaN,1.0,NaN
Texas_Tech,0.0,0.0,NaN,0.0,0.0,0.0,0.0,0.0,NaN,0.0


2018 lop 0.0
D


...

2018 lop 0.0
D


team2,Kansas,West_Virginia,Texas_Tech,Kansas_St,TCU,Texas,Oklahoma,Oklahoma_St,Baylor,Iowa_St
team1,,,,,,,,,,
Kansas,0.0,2.0,1.0,2.0,2.0,2.0,1.0,0.0,1.0,2.0
West_Virginia,0.0,0.0,0.0,2.0,1.0,1.0,2.0,1.0,2.0,1.0
Texas_Tech,0.0,0.0,0.0,2.0,2.0,1.0,1.0,1.0,1.0,1.0
Kansas_St,0.0,0.0,0.0,0.0,1.0,2.0,1.0,2.0,2.0,2.0
TCU,0.0,0.0,0.0,0.0,0.0,1.0,0.0,2.0,2.0,2.0
Texas,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0
Oklahoma,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
Oklahoma_St,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0
Baylor,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


2018 hillside 44
D


team2,Kansas,Kansas_St,West_Virginia,Texas_Tech,Texas,Oklahoma_St,Oklahoma,TCU,Iowa_St,Baylor
team1,,,,,,,,,,
Kansas,NaN,2.0,2.0,1.0,2.0,NaN,1.0,2.0,2.0,1.0
Kansas_St,0.0,NaN,0.0,0.0,2.0,2.0,1.0,1.0,2.0,2.0
West_Virginia,0.0,2.0,NaN,NaN,1.0,1.0,2.0,1.0,1.0,2.0
Texas_Tech,0.0,2.0,NaN,NaN,1.0,1.0,1.0,2.0,1.0,1.0
Texas,0.0,0.0,0.0,0.0,NaN,NaN,NaN,0.0,2.0,NaN
Oklahoma_St,NaN,0.0,0.0,0.0,NaN,NaN,NaN,0.0,2.0,NaN
Oklahoma,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,1.0,NaN
TCU,0.0,0.0,0.0,0.0,1.0,2.0,NaN,NaN,2.0,2.0
Iowa_St,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,0.0


ranking  \
year method                                                        
2012 colley    team2
TCU               5
Baylor            2
...   
     massey    team2
TCU               9
Baylor            2
...   
     lop       team1
TCU               5
Baylor            2
...   
     hillside  team1
TCU              10
Baylor            4
...   
2013 colley    team2
TCU              10
Baylor            6
...   
     massey    team2
TCU              10
Baylor            6
...   
     lop       team1
TCU              10
Baylor            6
...   
     hillside  team1
TCU               8
Baylor            7
...   
2014 colley    team2
TCU              10
Baylor            6
...   
     massey    team2
TCU              10
Baylor            6
...   
     lop       team1
TCU              10
Baylor            6
...   
     hillside  team1
TCU              10
Baylor            8
...   
2015 colley    team2
TCU               9
Baylor            5
...   
     massey    team2
TCU               9
Baylor            4
...   
     lop       team1
TCU               9
Baylor            4
...   
     hillside  team1
TCU              10
Baylor            5
...   
2016 colley    team2
TCU              10
Baylor            6
...   
     massey    team2
TCU              10
Baylor            6
...   
     lop       team1
TCU              10
Baylor            6
...   
     hillside  team1
TCU              10
Baylor            7
...   
2017 colley    team2
TCU               7
Baylor            3
...   
     massey    team2
TCU               7
Baylor            3
...   
     lop       team1
TCU               7
Baylor            4
...   
     hillside  team1
TCU               8
Baylor            3
...   
2018 colley    team2
TCU               5
Baylor            9
...   
     massey    team2
TCU               4
Baylor            6
...   
     lop       team1
TCU               5
Baylor            9
...   
     hillside  team1
TCU               8
Baylor           10
...   

                                                               r  \
year method                                                        
2012 colley    team2
TCU              0.539351
Baylor        ...   
     massey    team2
TCU              -9.735951
Baylor       ...   
     lop       team1
TCU              5.044555e+00
Baylor    ...   
     hillside  team1
TCU              0
Baylor           6
Io...   
2013 colley    team2
TCU              0.075409
Baylor        ...   
     massey    team2
TCU             -16.249760
Baylor       ...   
     lop       team1
TCU              1.755635e-11
Baylor    ...   
     hillside  team1
TCU              2
Baylor           3
Io...   
2014 colley    team2
TCU              0.090909
Baylor        ...   
     massey    team2
TCU             -16.000000
Baylor       ...   
     lop       team1
TCU              2.012574e-10
Baylor    ...   
     hillside  team1
TCU              0
Baylor           2
Io...   
2015 colley    team2
TCU              0.214120
Baylor        ...   
     massey    team2
TCU              -6.318053
Baylor       ...   
     lop       team1
TCU              1.000000e+00
Baylor    ...   
     hillside  team1
TCU              0
Baylor           5
Io...   
2016 colley    team2
TCU              0.109081
Baylor        ...   
     massey    team2
TCU             -13.436885
Baylor       ...   
     lop       team1
TCU              1.367801e-10
Baylor    ...   
     hillside  team1
TCU              0
Baylor           3
Io...   
2017 colley    team2
TCU              0.321181
Baylor        ...   
     massey    team2
TCU             -4.058844
Baylor        ...   
     lop       team1
TCU              2.502563e+00
Baylor    ...   
     hillside  team1
TCU              2
Baylor           7
Io...   
2018 colley    team2
TCU              0.532867
Baylor        ...   
     massey    team2
TCU               3.206859
Baylor       ...   
     lop       team1
TCU              4.712262e+00
Baylor    ...   
     hillside  team1
TCU              2
Bayl

In [272]:
inconsistent_results = get_results(inconsistent_games)
inconsistent_results

2018 lop 0.0
D


team2,Texas_Tech,Oklahoma,Oklahoma_St,Kansas_St,West_Virginia,Texas,TCU,Iowa_St,Kansas,Baylor
team1,,,,,,,,,,
Texas_Tech,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Oklahoma,0.0,0.0,1.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0
Oklahoma_St,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0
Kansas_St,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0
West_Virginia,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Texas,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
TCU,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Iowa_St,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0
Kansas,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


2018 hillside 0
D


team2,West_Virginia,Texas_Tech,Oklahoma,Texas,Oklahoma_St,Kansas_St,Kansas,Iowa_St,Baylor,TCU
team1,,,,,,,,,,
West_Virginia,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Texas_Tech,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Oklahoma,NaN,0.0,NaN,NaN,1.0,2.0,NaN,NaN,NaN,NaN
Texas,NaN,NaN,NaN,NaN,0.0,0.0,NaN,1.0,NaN,NaN
Oklahoma_St,NaN,NaN,0.0,1.0,NaN,NaN,NaN,1.0,NaN,NaN
Kansas_St,NaN,NaN,0.0,1.0,NaN,NaN,NaN,NaN,1.0,NaN
Kansas,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN
Iowa_St,NaN,NaN,NaN,0.0,0.0,NaN,1.0,NaN,1.0,NaN
Baylor,NaN,NaN,NaN,NaN,NaN,0.0,NaN,0.0,NaN,NaN


2018 lop 0.0
D


...

2018 lop 4.0
D


team2,Iowa_St,Baylor,Texas,Oklahoma,Oklahoma_St,TCU,Texas_Tech,Kansas_St,West_Virginia,Kansas
team1,,,,,,,,,,
Iowa_St,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0
Baylor,0.0,0.0,2.0,1.0,2.0,0.0,1.0,0.0,0.0,1.0
Texas,0.0,0.0,0.0,2.0,1.0,1.0,1.0,0.0,1.0,0.0
Oklahoma,0.0,1.0,0.0,0.0,1.0,2.0,1.0,1.0,0.0,1.0
Oklahoma_St,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,1.0,2.0
TCU,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0
Texas_Tech,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0
Kansas_St,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
West_Virginia,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0


2018 hillside 15
D


team2,Iowa_St,Baylor,Oklahoma,Texas,Oklahoma_St,West_Virginia,Texas_Tech,Kansas_St,Kansas,TCU
team1,,,,,,,,,,
Iowa_St,NaN,1.0,1.0,NaN,NaN,1.0,1.0,NaN,NaN,NaN
Baylor,0.0,NaN,1.0,2.0,2.0,NaN,1.0,NaN,1.0,NaN
Oklahoma,0.0,1.0,NaN,0.0,1.0,NaN,1.0,1.0,1.0,2.0
Texas,NaN,0.0,2.0,NaN,1.0,1.0,1.0,NaN,NaN,1.0
Oklahoma_St,NaN,0.0,1.0,1.0,NaN,1.0,1.0,NaN,2.0,NaN
West_Virginia,0.0,NaN,NaN,0.0,0.0,NaN,1.0,NaN,NaN,0.0
Texas_Tech,0.0,0.0,0.0,0.0,0.0,1.0,NaN,NaN,1.0,NaN
Kansas_St,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,0.0
Kansas,NaN,0.0,0.0,NaN,0.0,NaN,0.0,NaN,NaN,NaN


ranking  \
year method                                                        
2012 colley    team2
TCU               9
Baylor            8
...   
     massey    team2
TCU               3
Baylor            9
...   
     lop       team1
TCU               7
Baylor           10
...   
     hillside  team1
TCU              10
Baylor            9
...   
2013 colley    team2
TCU               3
Baylor            5
...   
     massey    team2
TCU               6
Baylor            4
...   
     lop       team1
TCU               4
Baylor            6
...   
     hillside  team1
TCU               7
Baylor            6
...   
2014 colley    team2
TCU              10
Baylor            3
...   
     massey    team2
TCU               5
Baylor            3
...   
     lop       team1
TCU               6
Baylor            3
...   
     hillside  team1
TCU              10
Baylor            3
...   
2015 colley    team2
TCU               1
Baylor            6
...   
     massey    team2
TCU               1
Baylor            5
...   
     lop       team1
TCU               1
Baylor            6
...   
     hillside  team1
TCU               2
Baylor            6
...   
2016 colley    team2
TCU               1
Baylor            3
...   
     massey    team2
TCU               2
Baylor            5
...   
     lop       team1
TCU               1
Baylor            5
...   
     hillside  team1
TCU               2
Baylor            4
...   
2017 colley    team2
TCU               4
Baylor            8
...   
     massey    team2
TCU               3
Baylor            8
...   
     lop       team1
TCU               3
Baylor            8
...   
     hillside  team1
TCU               4
Baylor            8
...   
2018 colley    team2
TCU               6
Baylor            2
...   
     massey    team2
TCU               5
Baylor            2
...   
     lop       team1
TCU               6
Baylor            2
...   
     hillside  team1
TCU              10
Baylor            2
...   

                                                               r  \
year method                                                        
2012 colley    team2
TCU              0.000000
Baylor        ...   
     massey    team2
TCU               0.000000
Baylor       ...   
     lop       team1
TCU              3.755127
Baylor        ...   
     hillside  team1
TCU              0
Baylor           1
Io...   
2013 colley    team2
TCU              0.671297
Baylor        ...   
     massey    team2
TCU              -1.294406
Baylor       ...   
     lop       team1
TCU              6.549348e+00
Baylor    ...   
     hillside  team1
TCU              3
Baylor           4
Io...   
2014 colley    team2
TCU              0.000000
Baylor        ...   
     massey    team2
TCU              0.000000
Baylor        ...   
     lop       team1
TCU              3.651025
Baylor        ...   
     hillside  team1
TCU              0
Baylor           7
Io...   
2015 colley    team2
TCU              0.812688
Baylor        ...   
     massey    team2
TCU              16.074558
Baylor       ...   
     lop       team1
TCU              8.585456e+00
Baylor    ...   
     hillside  team1
TCU              8
Baylor           4
Io...   
2016 colley    team2
TCU              0.759087
Baylor        ...   
     massey    team2
TCU               5.552844
Baylor       ...   
     lop       team1
TCU              9.000000e+00
Baylor    ...   
     hillside  team1
TCU              8
Baylor           6
Io...   
2017 colley    team2
TCU              0.618987
Baylor        ...   
     massey    team2
TCU               2.317023
Baylor       ...   
     lop       team1
TCU              6.573758
Baylor        ...   
     hillside  team1
TCU              6
Baylor           2
Io...   
2018 colley    team2
TCU              0.420691
Baylor        ...   
     massey    team2
TCU               1.029043
Baylor       ...   
     lop       team1
TCU              3.703253
Baylor        ...   
     hillside  team1
TCU              0
Bayl

In [233]:
import kendall_w as kw
def calc_kws(results):
    kws = []
    results_year = results.reset_index().set_index('year')
    for year in years:
        annotations = np.array([list(v.values.flat) for v in results_year.loc[year]['ranking']]).T
        kws.append(kw.compute_w(annotations.tolist()))
    kws = pd.Series(kws,index=years)
    return kws
kws = calc_kws(results)
kws

2012    0.900000
2013    0.948485
2014    0.896970
2015    0.953030
2016    0.972727
2017    0.963636
2018    0.865152
dtype: float64

### Now we will add in an upset and see how much the agreement changes

In [249]:
dt = 0
methods = ['colley','massey','lop','hillside']
results2 = pd.DataFrame(columns=['ranking','r','perm','year','method']).set_index(['year','method'])
ntimes_upset = 20
for year, team_range in teams_by_year.items():
    games_df = pd.DataFrame({"team1_name":consistent_games[year]['team1_name'],
                            "team1_score":consistent_games[year]['points1'],
                            "team1_H_A_N": consistent_games[year]['H_A_N1'],
                            "team2_name":consistent_games[year]['team2_name'],
                            "team2_score":consistent_games[year]['points2'],
                            "team2_H_A_N": consistent_games[year]['H_A_N1'],
                            "date": consistent_games[year]['date']
                           }).sort_values(by='date').drop('date',axis=1)    
    order_teams = percentages[year].sort_values().index
    top_team = order_teams[-1]
    last_team = order_teams[0]
    print(top_team,last_team)
    for niter in range(ntimes_upset):
        last_team_score = max(max(games_df['team1_score']),max(games_df['team2_score'])) # make this a big upset win
        top_team_score = min(min(games_df['team1_score']),min(games_df['team2_score']))
        #top_team_score,last_team_score = last_team_score,top_team_score
        games_df = games_df.append(pd.Series([top_team,top_team_score,1,last_team,last_team_score,-1],index=games_df.columns,name=-(niter+1)))
    for method in methods:
        ranking,r,perm = compute(dt,games_df,team_range,method)
        entry = pd.Series([ranking,r,perm],index=['ranking','r','perm'],name=(year,method))
        results2 = results2.append(entry)

TCU Texas_Tech
2012 lop 1.0
D


team2,Kansas,Baylor,Iowa_St,West_Virginia,Texas,Kansas_St,Oklahoma_St,Oklahoma,Texas_Tech,TCU
team1,,,,,,,,,,
Kansas,0.0,2.0,1.0,0.0,2.0,2.0,2.0,2.0,2.0,0.0
Baylor,0.0,0.0,1.0,1.0,2.0,1.0,2.0,2.0,2.0,0.0
Iowa_St,0.0,0.0,0.0,0.0,1.0,2.0,1.0,2.0,2.0,0.0
West_Virginia,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
Texas,0.0,0.0,0.0,0.0,0.0,1.0,1.0,2.0,2.0,0.0
Kansas_St,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,2.0,0.0
Oklahoma_St,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,2.0,0.0
Oklahoma,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
Texas_Tech,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,20.0


2012 hillside 5
D


team2,Kansas,Iowa_St,West_Virginia,Baylor,Kansas_St,Texas,Oklahoma_St,Oklahoma,Texas_Tech,TCU
team1,,,,,,,,,,
Kansas,NaN,1.0,NaN,2.0,2.0,2.0,2.0,2.0,2.0,NaN
Iowa_St,0.0,NaN,NaN,0.0,2.0,1.0,1.0,2.0,2.0,NaN
West_Virginia,NaN,NaN,NaN,0.0,1.0,NaN,NaN,NaN,NaN,NaN
Baylor,0.0,1.0,1.0,NaN,1.0,2.0,2.0,2.0,2.0,NaN
Kansas_St,0.0,0.0,0.0,0.0,NaN,0.0,2.0,NaN,2.0,NaN
Texas,0.0,0.0,NaN,0.0,1.0,NaN,1.0,2.0,2.0,NaN
Oklahoma_St,0.0,0.0,NaN,0.0,0.0,0.0,NaN,1.0,2.0,NaN
Oklahoma,0.0,0.0,NaN,0.0,NaN,0.0,0.0,NaN,1.0,NaN
Texas_Tech,0.0,0.0,NaN,0.0,0.0,0.0,0.0,0.0,NaN,20.0


Kansas_St TCU
2013 lop 14.0
D


...

team2,Kansas,Iowa_St,Baylor,West_Virginia,Oklahoma_St,Kansas_St,Texas_Tech,TCU,Texas,Oklahoma
team1,,,,,,,,,,
Kansas,NaN,1.0,2.0,1.0,2.0,2.0,2.0,2.0,2.0,2.0
Iowa_St,0.0,NaN,NaN,NaN,2.0,2.0,2.0,1.0,1.0,2.0
Baylor,0.0,NaN,NaN,NaN,2.0,1.0,1.0,2.0,2.0,2.0
West_Virginia,0.0,NaN,NaN,NaN,1.0,1.0,1.0,2.0,2.0,1.0
Oklahoma_St,0.0,0.0,0.0,0.0,NaN,1.0,2.0,2.0,1.0,2.0
Kansas_St,0.0,0.0,0.0,0.0,0.0,NaN,1.0,1.0,2.0,1.0
Texas_Tech,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,1.0,1.0
TCU,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,2.0,1.0
Texas,20.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,0.0


Kansas Iowa_St
2018 lop 13.0
D


team2,West_Virginia,Texas_Tech,Kansas_St,TCU,Oklahoma_St,Oklahoma,Texas,Baylor,Iowa_St,Kansas
team1,,,,,,,,,,
West_Virginia,0.0,0.0,2.0,1.0,1.0,2.0,1.0,2.0,1.0,0.0
Texas_Tech,0.0,0.0,2.0,2.0,1.0,1.0,1.0,1.0,1.0,0.0
Kansas_St,0.0,0.0,0.0,1.0,2.0,1.0,2.0,2.0,2.0,0.0
TCU,0.0,0.0,0.0,0.0,2.0,0.0,1.0,2.0,2.0,0.0
Oklahoma_St,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0
Oklahoma,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
Texas,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0
Baylor,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
Iowa_St,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,20.0


2018 hillside 58
D


team2,Kansas,Kansas_St,West_Virginia,Texas_Tech,Texas,Oklahoma_St,Oklahoma,TCU,Iowa_St,Baylor
team1,,,,,,,,,,
Kansas,NaN,2.0,2.0,1.0,2.0,NaN,1.0,2.0,2.0,1.0
Kansas_St,0.0,NaN,0.0,0.0,2.0,2.0,1.0,1.0,2.0,2.0
West_Virginia,0.0,2.0,NaN,NaN,1.0,1.0,2.0,1.0,1.0,2.0
Texas_Tech,0.0,2.0,NaN,NaN,1.0,1.0,1.0,2.0,1.0,1.0
Texas,0.0,0.0,0.0,0.0,NaN,NaN,NaN,0.0,2.0,NaN
Oklahoma_St,NaN,0.0,0.0,0.0,NaN,NaN,NaN,0.0,2.0,NaN
Oklahoma,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,1.0,NaN
TCU,0.0,0.0,0.0,0.0,1.0,2.0,NaN,NaN,2.0,2.0
Iowa_St,20.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,0.0


In [250]:
from scipy import stats
taus = []

for index in results.index:
    index
    ranking = results.loc[index,'ranking']
    ranking2 = results2.loc[index,'ranking']
    tau, p_value = stats.kendalltau(ranking, ranking2)
    taus.append(tau)
taus = pd.Series(taus,results.index,name='tau')
taus = taus.to_frame().reset_index()
taus

,year,method,tau
0,2012,colley,0.733333
1,2012,massey,0.955556
2,2012,lop,0.777778
3,2012,hillside,1.000000
4,2013,colley,0.600000
5,2013,massey,0.333333
6,2013,lop,0.600000
7,2013,hillside,1.000000
8,2014,colley,0.511111
9,2014,massey,0.111111


In [251]:
import altair as alt

alt.Chart(taus).mark_bar().encode(
    x='method',
    y=alt.Y('tau'),
    color='method',
    column='year'
)

alt.Chart(...)

In [168]:
taus.groupby('year').mean()

,tau
year,
2002,0.755556
2003,0.855556
2004,0.811111
2005,0.833333
2006,0.866667
2007,0.844444
2008,0.844444
2009,0.877778
2010,0.833333


In [165]:
kws2 = calc_kws(results2)
kws2

2002    0.943939
2003    0.913636
2004    0.936364
2005    0.936364
2006    0.846970
2007    0.904545
2008    0.862121
2009    0.956061
2010    0.904545
2011    0.915152
2012    0.937879
2013    0.925758
2014    0.946970
2015    0.943939
2016    0.971212
2017    0.922727
2018    0.907576
dtype: float64

In [166]:
kw_df = pd.DataFrame({'V0':kws,'V1':kws2})
kw_df

,V0,V1
2002,0.912121,0.943939
2003,0.877273,0.913636
2004,0.877273,0.936364
2005,0.883333,0.936364
2006,0.889394,0.846970
2007,0.854545,0.904545
2008,0.901515,0.862121
2009,0.919697,0.956061
2010,0.898485,0.904545
2011,0.878788,0.915152


In [167]:
100*(kws - kws2)/kws

2002   -3.488372
2003   -4.145078
2004   -6.735751
2005   -6.003431
2006    4.770017
2007   -5.851064
2008    4.369748
2009   -3.953871
2010   -0.674536
2011   -4.137931
2012   -1.143791
2013    5.709877
2014    0.000000
2015    1.735016
2016    1.536098
2017    4.245283
2018    2.442997
dtype: float64